In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import timm
import torch
import os
import tabulate
import db_utils as dbu

device = 'cpu' # torch.device('cuda' if torch.cuda.is_available() else 'cpu')
basepath = os.path.abspath('../figures/key-value-and-class-value-agreement')
os.makedirs(basepath, exist_ok=True)

model_names = ['vit_base_patch16_224', 
               'vit_base_patch16_224_miil', 
               'vit_base_patch32_224', 
               'vit_large_patch16_224']
pretty_model_names = {
    'vit_base_patch16_224': 'ViT-B/16', 
    'vit_base_patch16_224_miil': 'ViT-B/16-MIIL', 
    'vit_base_patch32_224': 'ViT-B/32', 
    'vit_large_patch16_224': 'ViT-L/16',
    'imagenet_val': 'ImageNet1k-Val'
}
models = {
    model_name: timm.create_model(model_name, pretrained=True).eval().to(device)
    for model_name in model_names
}

In [2]:
transferability_db = "A:\\CVData\\project-images\\images-mixed-1280-fc1-multimodel-maco\\transferability.db"
plausbility_db = "A:\\CVData\\project-images\\images-mixed-1280-fc1-multimodel-maco\\plausibility_with_gen.db"
fid_top1_path = "A:\CVData\project-images\images-mixed-1280-fc1-multimodel-maco\fid_top1_corrected.json"
fid_top1234_sm_path = "A:\CVData\project-images\images-mixed-1280-fc1-multimodel-maco\fid_top1234_sm_corrected.json"
fid_top_1234_uw_path = "A:\CVData\project-images\images-mixed-1280-fc1-multimodel-maco\fid_top1234_uw_corrected.json"

In [15]:
from src.utils.extraction import extract_value_vectors
from src.utils.model import embedding_projection
from src.analyzers.vector_analyzer import most_predictive_ind_for_classes
from src.utils.imagenet import get_name_for_index

distances_by_cat = dbu.double_group_by_tensor(
    dbu.select(plausbility_db, "select gen_model, num_idx, distance from distances where `top`='1'"),
    0, 1, 2, True)

projected_values = {
    model_name: embedding_projection(models[model_name], 
                                     extract_value_vectors(models[model_name], device), device)
    for model_name in model_names
}
most_pred_inds = {
    model_name: most_predictive_ind_for_classes(projected_values[model_name], device)
    for model_name in model_names
}

scores_by_block = {
    model_name: [torch.inf for _ in range(len(models[model_name].blocks))]
    for model_name in model_names
}

for model_name in model_names:
    for block in range(len(models[model_name].blocks)):
        classes_of_block = most_pred_inds[model_name][
            2,most_pred_inds[model_name][0,:]==block].tolist()
        tensors = [distances_by_cat[model_name][cls] for cls in classes_of_block]
        if len(tensors) > 0:
            scores_by_block[model_name][block] = torch.stack(tensors).mean().item()
    scores_by_block[model_name] = dbu.collect_into_df(
        torch.tensor(scores_by_block[model_name]))

scores_by_block[model_name]

{'vit_base_patch16_224': [inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  1584.205078125,
  1454.67919921875,
  998.8911743164062,
  997.125,
  1411.4940185546875],
 'vit_base_patch16_224_miil': [inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  409.57373046875,
  332.88092041015625,
  325.1775207519531,
  366.90228271484375,
  609.9409790039062],
 'vit_base_patch32_224': [inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  356.0640869140625,
  396.770263671875,
  312.05731201171875,
  428.39447021484375],
 'vit_large_patch16_224': [inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  inf,
  1376.640625,
  1374.631103515625,
  1306.1536865234375,
  1223.7996826171875,
  1158.329345703125,
  1130.003173828125,
  1158.061279296875,
  1217.5819091796875,
  1334.975830078125,
  1430.1588134765625]}

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")